In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
salaries = pd.read_csv('data/salaries_data.csv')

In [3]:
salaries.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2022,SE,FT,Data Engineer,140250,USD,140250,US,100,US,M
1,2022,SE,FT,Data Engineer,135000,USD,135000,US,100,US,M
2,2021,MI,FT,BI Data Analyst,100000,USD,100000,US,100,US,M
3,2021,MI,CT,ML Engineer,270000,USD,270000,US,100,US,L
4,2021,MI,FT,Data Engineer,22000,EUR,26005,RO,0,US,L


In [4]:
test = pd.read_csv('data/testeo.csv')

In [5]:
test.head()

,work_year,experience_level,employment_type,job_title,employee_residence,remote_ratio,company_location,company_size
0,2020,SE,FT,Machine Learning Scientist,JP,0,JP,S
1,2020,MI,FT,Lead Data Analyst,US,100,US,L
2,2020,MI,FT,Data Analyst,US,100,US,L
3,2020,MI,FT,Machine Learning Engineer,CN,0,CN,M
4,2020,MI,FT,Product Data Analyst,IN,100,IN,L


In [6]:
len(salaries.columns)==len(test.columns)

False

In [7]:
salaries.drop(columns=salaries[['salary', 'salary_currency']], axis=1, inplace=True)

In [8]:
len(salaries.columns)

9

In [9]:
salaries.columns

Index(['work_year', 'experience_level', 'employment_type', 'job_title',
       'salary_in_usd', 'employee_residence', 'remote_ratio',
       'company_location', 'company_size'],
      dtype='object')

In [10]:
test.columns

Index(['work_year', 'experience_level', 'employment_type', 'job_title',
       'employee_residence', 'remote_ratio', 'company_location',
       'company_size'],
      dtype='object')

In [11]:
len(test.columns)

8

In [12]:
pip install h2o

Note: you may need to restart the kernel to use updated packages.


In [14]:
import h2o
from h2o.automl import H2OAutoML

In [15]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.281-b09, mixed mode)
  Starting server from C:\Users\Alvaro\anaconda3\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\Alvaro\AppData\Local\Temp\tmpjn6xjvgs
  JVM stdout: C:\Users\Alvaro\AppData\Local\Temp\tmpjn6xjvgs\h2o_Alvaro_started_from_python.out
  JVM stderr: C:\Users\Alvaro\AppData\Local\Temp\tmpjn6xjvgs\h2o_Alvaro_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,07 secs
H2O_cluster_timezone:,Europe/Paris
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.1
H2O_cluster_version_age:,24 days
H2O_cluster_name:,H2O_from_python_Alvaro_6ysj9n
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.521 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [16]:
h2train=h2o.H2OFrame(salaries)
h2test=h2o.H2OFrame(test)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [37]:
h2train.columns

['work_year',
 'experience_level',
 'employment_type',
 'job_title',
 'salary_in_usd',
 'employee_residence',
 'remote_ratio',
 'company_location',
 'company_size']

In [17]:
X=[c for c in h2train.columns if c!='salary_in_usd']

y='salary_in_usd'


In [18]:
automl=H2OAutoML(max_models=50,
                 seed=42, 
                 max_runtime_secs=300,
                 sort_metric='RMSE')

In [19]:
automl.train(x=X,
             y=y,
             training_frame=h2train)

AutoML progress: |
18:25:58.966: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_grid_1_AutoML_1_20230305_182558_model_2


Status of Neuron Layers: predicting salary_in_usd, regression, gaussian distribution, Quadratic loss, 16.101 weights/biases, 197,1 KB, 1.225.000 training samples, mini-batch size 1
    layer    units    type              dropout    l1    l2    mean_rate              rate_rms               momentum    mean_weight            weight_rms           mean_bias              bias_rms
--  -------  -------  ----------------  ---------  ----  ----  ---------------------  ---------------------  ----------  ---------------------  -------------------  ---------------------  -----------------------
    1        159      Input             0.0
    2        100      RectifierDropout  20.0       0.0   0.0   0.049761964527038204   0.20826101303100586    0.0         -0.012346447598718472  0.11824426054954529  0.15374788108021564    0.13196247816085815
    3        1        Linear                       0.0   0.0   0.0005445577771752141  0.0003911318490281701  0.0         -0.001130193006247282  0.29311704635620117  0.0010153357505624815  1.0971281125650402e-154

ModelMetricsRegression: deeplearning
** Reported on train data. **

MSE: 641467619.5099168
RMSE: 25327.21104878934
MAE: 15783.499612950123
RMSLE: 0.21500315207770726
Mean Residual Deviance: 641467619.5099168

ModelMetricsRegression: deeplearning
** Reported on cross-validation data. **

MSE: 2369855171.0828905
RMSE: 48681.158275896545
MAE: 32807.43098301167
RMSLE: NaN
Mean Residual Deviance: 2369855171.0828905

Cross-Validation Metrics Summary: 
                        mean         sd           cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid
----------------------  -----------  -----------  ------------  ------------  ------------  ------------  ------------
mae                     32807.4      3190.98      34087.2       29769.2       29401.7       33820         36959.1
mean_residual_deviance  2.36986e+09  8.05041e+08  3.07507e+09   1.63005e+09   1.46081e+09   2.47009e+09   3.21325e+09
mse                     2.36986e+09  8.05041e+08  3.07507e+09   1.63005e+09   1.46081e+09   2.47009e+09   3.21325e+09
r2                      0.514089     0.0837104    0.463545      0.612536      0.54756       0.548742      0.398063
residual_deviance       2.36986e+09  8.05041e+08  3.07507e+09   1.63005e+09   1.46081e+09   2.47009e+09   3.21325e+09
rmse                    48086.7      8479.95      55453.3       40373.8       38220.6       49700.1       56685.5
rmsle                   0.58114      0.0166192    nan           0.592891      nan           nan           0.569388

Scoring History: 
    timestamp            duration          training_speed    epochs    iterations    samples    training_rmse    training_deviance    training_mae    training_r2
--  -------------------  ----------------  ----------------  --------  ------------  ---------  ---------------  -------------------  --------------  -------------
    2023-03-05 18:27:54  0.000 sec                           0         0             0          nan              nan                  nan             nan
    2023-03-05 18:27:54  1 min 44.885 sec  32051 obs/sec     10        1             5000       52088.2          2.71318e+09          34165.7         0.439007
    2023-03-05 18:27:59  1 min 49.887 sec  70818 obs/sec     730       73            365000     30848.6          9.51637e+08          19683.6         0.803234
    2023-03-05 18:28:04  1 min 54.907 sec  74729 obs/sec     1520      152           760000     26789            7.1765e+08           17128           0.851614
    2023-03-05 18:28:09  1 min 59.937 sec  77986 obs/sec     2370      237           1.185e+06  25343.4          6.42288e+08          15824.6         0.867197
    2023-03-05 18:28:10  2 min  0.433 sec  78100 obs/sec     2450      245           1.225e+06  25327.2          6.41468e+08          15783.5         0.867366

Variable Importan

In [20]:
print('[INFO] Leader board:')

leader_board=automl.leaderboard

leader_board.head()

[INFO] Leader board:


model_id,rmse,mse,mae,rmsle,mean_residual_deviance
DeepLearning_grid_1_AutoML_1_20230305_182558_model_2,48681.2,2.36986e+09,32807.4,nan,2.36986e+09
DeepLearning_grid_1_AutoML_1_20230305_182558_model_3,49133.5,2.41411e+09,33357.3,nan,2.41411e+09
GBM_grid_1_AutoML_1_20230305_182558_model_12,49729.4,2.47301e+09,32938.1,0.514271,2.47301e+09
DRF_1_AutoML_1_20230305_182558,49755.3,2.47559e+09,32062.8,0.487596,2.47559e+09
DeepLearning_1_AutoML_1_20230305_182558,50347.3,2.53485e+09,35569,nan,2.53485e+09
GBM_3_AutoML_1_20230305_182558,50359,2.53603e+09,32193.6,0.512161,2.53603e+09
GBM_grid_1_AutoML_1_20230305_182558_model_4,50612,2.56158e+09,32429.4,0.518104,2.56158e+09
GBM_5_AutoML_1_20230305_182558,50738.1,2.57435e+09,32373,0.511648,2.57435e+09
GBM_4_AutoML_1_20230305_182558,50760,2.57658e+09,32697.7,0.51385,2.57658e+09
GBM_grid_1_AutoML_1_20230305_182558_model_1,50762.4,2.57682e+09,32458.2,0.513602,2.57682e+09


In [21]:
y_pred=automl.leader.predict(h2test)

deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%


C:\Users\Alvaro\anaconda3\lib\site-packages\h2o\job.py:83: UserWarning: Test/Validation dataset column 'employee_residence' has levels not trained on: ["AR", "CN", "CO", "DK", "HR", "IQ", "PH", "TN"]
  warnings.warn(w)
C:\Users\Alvaro\anaconda3\lib\site-packages\h2o\job.py:83: UserWarning: Test/Validation dataset column 'job_title' has levels not trained on: ["NLP Engineer", "Principal Data Engineer"]
  warnings.warn(w)
C:\Users\Alvaro\anaconda3\lib\site-packages\h2o\job.py:83: UserWarning: Test/Validation dataset column 'company_location' has levels not trained on: ["CO", "HR", "IL", "IQ", "IT", "MY", "RU"]
  warnings.warn(w)


In [27]:
solution5= y_pred.as_data_frame()

In [28]:
solution5

,predict
0,151541.335556
1,123685.408286
2,96947.759174
3,65149.623440
4,36412.693103
...,...
102,108907.972469
103,212977.469805
104,164260.327417
105,66766.680251


In [29]:
solution5.head()

,predict
0,151541.335556
1,123685.408286
2,96947.759174
3,65149.623440
4,36412.693103


In [31]:
solution5.reset_index(drop=True, inplace=True)

In [32]:
solution5

,predict
0,151541.335556
1,123685.408286
2,96947.759174
3,65149.623440
4,36412.693103
...,...
102,108907.972469
103,212977.469805
104,164260.327417
105,66766.680251


In [34]:
solution5.rename(columns={ 'predict': "salary_in_usd"}, inplace= True)

In [35]:
solution5

,salary_in_usd
0,151541.335556
1,123685.408286
2,96947.759174
3,65149.623440
4,36412.693103
...,...
102,108907.972469
103,212977.469805
104,164260.327417
105,66766.680251


In [36]:
solution5.to_csv('sols/sol5.csv')

### 